In [ ]:
import pandas as pd
from tqdm import tqdm
import gc
import numpy as np
import lightgbm as lgb
# from calc_uauc import uAUC
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from scipy import sparse
import os
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', 100)
tqdm.pandas(desc='pandas bar')


def get_count(df, column, feature):
    df['idx'] = range(len(df))
    temp = df.groupby(column)['userid'].agg([(feature, 'count')]).reset_index()
    df = df.merge(temp)
    df = df.sort_values('idx').drop('idx', axis=1).reset_index(drop=True)
    return df


def get_sum(df, column, feature):
    df['idx'] = range(len(df))
    temp = df.groupby(column)['userid'].agg([(feature, 'sum')]).reset_index()
    df = df.merge(temp)
    df = df.sort_values('idx').drop('idx', axis=1).reset_index(drop=True)
    return df


def reduce_mem(df):
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    print('{:.2f} Mb, {:.2f} Mb ({:.2f} %)'.format(start_mem, end_mem, 100 * (start_mem - end_mem) / start_mem))
    gc.collect()
    return df

In [ ]:
train = pd.read_csv('../data/train/05-17.csv', header=None)
train.columns = ['time','gun_no','W_Error','C_Cylinder_force','C_Differential_pressure','W_Friction','W_Maximum_aperture','W_Maximum_electrode_force','W_Start_friction','W_us2','W_Welding_point_count','W_position_count','area','in_Counterbalance_pressure','in_Electrode_force','in_Electrode_position','in_Sheet_thickness',
'in_Velocity','in_force_build_up','out_Cap_offset','out_Electrode_force','out_Electrode_position','out_force_build_up']
train = reduce_mem(train)

train1 = pd.read_csv('../data/train/05-18.csv', header=None)
train1.columns = ['time','gun_no','W_Error','C_Cylinder_force','C_Differential_pressure','W_Friction','W_Maximum_aperture','W_Maximum_electrode_force','W_Start_friction','W_us2','W_Welding_point_count','W_position_count','area','in_Counterbalance_pressure','in_Electrode_force','in_Electrode_position','in_Sheet_thickness',
'in_Velocity','in_force_build_up','out_Cap_offset','out_Electrode_force','out_Electrode_position','out_force_build_up']
train1 = reduce_mem(train1)

test = pd.read_csv('../data/phm_test.csv', header=None)
test.columns = ['time','gun_no','C_Cylinder_force','C_Differential_pressure','W_Friction','W_Maximum_aperture','W_Maximum_electrode_force','W_Start_friction','W_us2','W_Welding_point_count','W_position_count','area','in_Counterbalance_pressure','in_Electrode_force','in_Electrode_position','in_Sheet_thickness',
'in_Velocity','in_force_build_up','out_Cap_offset','out_Electrode_force','out_Electrode_position','out_force_build_up']
test = reduce_mem(test)

186.84 Mb, 67.02 Mb (64.13 %)
186.79 Mb, 67.00 Mb (64.13 %)
503.54 Mb, 165.94 Mb (67.05 %)


In [ ]:
train.head(5)

,time,gun_no,W_Error,C_Cylinder_force,C_Differential_pressure,W_Friction,W_Maximum_aperture,W_Maximum_electrode_force,W_Start_friction,W_us2,W_Welding_point_count,W_position_count,area,in_Counterbalance_pressure,in_Electrode_force,in_Electrode_position,in_Sheet_thickness,in_Velocity,in_force_build_up,out_Cap_offset,out_Electrode_force,out_Electrode_position,out_force_build_up
0,1621209600,6015660-3105,0,351,0.429932,63,285.000,4763,359,1,3372415,17089160,F2,15.992188,3700,80.0,3.000000,1,0,0.000000,0,78.812500,0
1,1621209600,5509057-3050,0,-34,1.019531,22,157.125,6598,202,1,374308,22730778,F3,14.992188,3300,25.0,3.099609,1,0,1.500000,0,2.000000,0
2,1621209600,5509057-3052,0,3230,-4.671875,-11,159.750,6074,265,1,374317,2125961,F3,8.046875,3200,25.0,3.099609,1,1,0.000000,3200,1.900391,1
3,1621209600,6015650-3010,0,-33,0.520020,33,307.000,4350,244,1,42961,20749108,F3,15.992188,2200,100.0,1.500000,1,0,2.699219,0,99.875000,0
4,1621209600,5507537-1010,0,-101,1.000000,-180,158.125,5147,130,1,2690730,15769638,FE,14.992188,2000,150.0,1.400391,1,0,1.299805,0,149.250000,0


In [ ]:
train['W_Error'].value_counts()

0       769867
0       210196
E011     45913
E003     38764
E016        39
Name: W_Error, dtype: int64

In [ ]:
# data = pd.concat([train, train1], axis=0)

train['W_Error'][train['W_Error']==0] = '0'
# print(train['W_Error'].value_counts())
# train['task1_label'] = 1
train['task1_label'] = train['W_Error'].apply(lambda x:0 if x=='0' else 1)
print(train['task1_label'].value_counts())

train1['W_Error'][train1['W_Error']==0] = '0'
# print(train['W_Error'].value_counts())
# train['task1_label'] = 1
train1['task1_label'] = train1['W_Error'].apply(lambda x:0 if x=='0' else 1)
print(train1['task1_label'].value_counts())

train['task2_label'] = train['W_Error'].apply(lambda x:1 if x=='E016' else 0)
train['task2_label'].value_counts()

0    980063
1     84716
Name: task1_label, dtype: int64
0    960768
1    103717
Name: task1_label, dtype: int64


0    1064740
1         39
Name: task2_label, dtype: int64

In [ ]:
drop_columns = ['time', 'gun_no','area']
train.drop(drop_columns, axis=1, inplace=True)
test.drop(drop_columns, axis=1, inplace=True)
train1.drop(drop_columns, axis=1, inplace=True)
train.head(5)

,W_Error,C_Cylinder_force,C_Differential_pressure,W_Friction,W_Maximum_aperture,W_Maximum_electrode_force,W_Start_friction,W_us2,W_Welding_point_count,W_position_count,in_Counterbalance_pressure,in_Electrode_force,in_Electrode_position,in_Sheet_thickness,in_Velocity,in_force_build_up,out_Cap_offset,out_Electrode_force,out_Electrode_position,out_force_build_up,task1_label,task2_label
0,0,351,0.429932,63,285.000,4763,359,1,3372415,17089160,15.992188,3700,80.0,3.000000,1,0,0.000000,0,78.812500,0,0,0
1,0,-34,1.019531,22,157.125,6598,202,1,374308,22730778,14.992188,3300,25.0,3.099609,1,0,1.500000,0,2.000000,0,0,0
2,0,3230,-4.671875,-11,159.750,6074,265,1,374317,2125961,8.046875,3200,25.0,3.099609,1,1,0.000000,3200,1.900391,1,0,0
3,0,-33,0.520020,33,307.000,4350,244,1,42961,20749108,15.992188,2200,100.0,1.500000,1,0,2.699219,0,99.875000,0,0,0
4,0,-101,1.000000,-180,158.125,5147,130,1,2690730,15769638,14.992188,2000,150.0,1.400391,1,0,1.299805,0,149.250000,0,0,0


In [ ]:
# w_error = train.pop('W_Error')
# task1_label = train.pop('task1_label')

In [ ]:
# train, y_train, X_test, y_test = train_test_split(train, train['task1_label'])
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=2019)
oof_lgb = np.zeros(len(train))
feature_importance_df = pd.DataFrame()
col = [tmp_col for tmp_col in train.columns if tmp_col not in ['W_Error', 'task1_label', 'task2_label']]
X_train = train[col].values
# print(train[col].info())
y_train = train['task1_label'].values
prediction = np.zeros(len(train1))

In [ ]:
param = {'num_leaves': 48,
         # 'min_data_in_leaf': 30,
         'objective': 'binary',
         'learning_rate': 0.02,
         # 'max_depth': 9,
         # "min_child_samples": 20,
         "boosting": "gbdt",
         "feature_fraction": 0.9,
         # "bagging_freq": 1,
         "bagging_fraction": 0.9,
         # "bagging_seed": 11,
         "metric": 'auc',
         # "lambda_l1": 0.1,
         # "lambda_l2": 0.1,
         "verbosity": -1}


In [ ]:
for fold_, (trn_idx, val_idx) in enumerate(folds.split(X_train, y_train)):
    print("fold n°{}".format(fold_ + 1))
    trn_data = lgb.Dataset(X_train[trn_idx], y_train[trn_idx])
    val_data = lgb.Dataset(X_train[val_idx], y_train[val_idx])

    num_round = 2000
    clf = lgb.train(param, trn_data, num_round, valid_sets=[trn_data, val_data], verbose_eval=200,
                    early_stopping_rounds=100)
    oof_lgb[val_idx] = clf.predict(X_train[val_idx], num_iteration=clf.best_iteration)
    prediction += clf.predict(train1[col].values, num_iteration=clf.best_iteration) / folds.n_splits

    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = col
    fold_importance_df["importance"] = clf.feature_importance(importance_type='gain')
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
# oof_lgb2 += clf.predict(df_remove[col].values, num_iteration=clf.best_iteration)
# oof_lgb_final = np.argmax(oof_lgb, axis=1)
#     predictions_lgb += clf.predict(X_test, num_iteration=clf.best_iteration) / folds.n_splits
print('feature importance:')
print(feature_importance_df.groupby(['feature'])['importance'].mean().sort_values(ascending=False).head(15))

fold n°1
Training until validation scores don't improve for 100 rounds
[200]	training's auc: 1	valid_1's auc: 1
[400]	training's auc: 1	valid_1's auc: 1
Early stopping, best iteration is:
[302]	training's auc: 1	valid_1's auc: 1
fold n°2
Training until validation scores don't improve for 100 rounds
[200]	training's auc: 1	valid_1's auc: 1
Early stopping, best iteration is:
[260]	training's auc: 1	valid_1's auc: 1
fold n°3
Training until validation scores don't improve for 100 rounds
[200]	training's auc: 1	valid_1's auc: 1
Early stopping, best iteration is:
[192]	training's auc: 1	valid_1's auc: 1
fold n°4
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[30]	training's auc: 1	valid_1's auc: 1
fold n°5
Training until validation scores don't improve for 100 rounds
[200]	training's auc: 1	valid_1's auc: 1
Early stopping, best iteration is:
[193]	training's auc: 1	valid_1's auc: 1
feature importance:
feature
in_Counterbalance_pressure    5.4

In [ ]:
label_lgb = np.where(oof_lgb > 0.3, 1, 0)

label_test = np.where(prediction > 0.3, 1, 0)

In [ ]:
def f2_score(y_true, y_pred):
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    print(precision, recall)
    f2_score = 5 * precision * recall / (4 * precision + recall)
    return f2_score
print('f1_score::{}'.format(f1_score(y_train, label_lgb)))
print(f2_score(y_train, label_lgb))
print(f2_score(train1['task1_label'].values, label_test))
# print('test_auc:{}'.format(roc_auc_score(y_test, prediction)))
print(pd.DataFrame(label_lgb)[0].value_counts())

f1_score::0.9997875504567666
0.9996695619335347 0.9999055668350725
0.9998583569405102
0.995051279498158 0.9635161063277958
0.9696622148737234
0    980043
1     84736
Name: 0, dtype: int64
